In [1]:
import datetime
import os.path
import sys

import pandas as pd
import numpy as np

In [2]:
!head ../data/raw/ES2006tick.csv

In [3]:
path = '../data/raw/ES2006tick.csv'
parse = lambda x: datetime.datetime.strptime(x[:-1],'%Y%m%d %H:%M:%S.%f')
tick_data = pd.read_csv(path, usecols=['Date and Time', 'Close', 'Volume'], 
                        index_col='Date and Time', parse_dates=['Date and Time'],
                        date_parser=parse,
                        dtype={'Close': np.float32, 'Volume': np.int16})
##Convert to EST
tick_data.index = tick_data.index.tz_localize('US/Eastern')
#tick_data = tick_data.tz_convert('US/Eastern')

##Save to feather format -- need to make index a column (won't take the datetime)
#tick_data.reset_index(inplace=True)
tick_data.columns = ['last','volume']
tick_data.index.rename('date', inplace=True)
#tick_data.to_feather('../data/processed/ES_all_tick.feather')

store = pd.HDFStore('../data/processed/store.h5')
store.put('tick_data',tick_data)
store.close()

tick_data.head()

,last,volume
date,,
2006-01-02 17:00:02-05:00,1255.25,2
2006-01-02 17:00:04-05:00,1255.50,3
2006-01-02 17:00:05-05:00,1255.25,1
2006-01-02 17:00:08-05:00,1255.50,5
2006-01-02 17:00:08-05:00,1255.50,5


In [ ]:
td = pd.read_feather('../data/processed/ES_all_tick.feather')

In [11]:
store.close()

In [2]:
td = pd.read_hdf('../data/processed/store.h5', key='tick_data')
td.head()

,last,volume
date,,
2006-01-02 17:00:02-05:00,1255.25,2
2006-01-02 17:00:04-05:00,1255.50,3
2006-01-02 17:00:05-05:00,1255.25,1
2006-01-02 17:00:08-05:00,1255.50,5
2006-01-02 17:00:08-05:00,1255.50,5


In [3]:
td['dv'] = td['last'] * td['volume']

In [4]:
td.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 918971351 entries, 2006-01-02 22:00:02-05:00 to 2018-06-22 19:59:59.802000-04:00
Data columns (total 3 columns):
last      float32
volume    int16
dv        float32
dtypes: float32(2), int16(1)
memory usage: 35.4 GB


In [6]:
td = td.astype({'last': np.float16, 'volume': np.int16, 'dv': np.float16})
td.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 918971351 entries, 2006-01-02 22:00:02-05:00 to 2018-06-22 19:59:59.802000-04:00
Data columns (total 3 columns):
last      float16
volume    int16
dv        float16
dtypes: float16(2), int16(1)
memory usage: 32.0 GB


In [8]:
store = pd.HDFStore('../data/processed/store.h5')
store.put('tick_data_dv2_small',st)
store.close()

In [7]:
tick_data = tick_data.set_index('Date')

In [7]:
st = td[td.index > '2017-12-31']
st.head()

,last,volume,dv
date,,,
2018-01-01 17:00:00.087000-05:00,2675.25,680,1819170.00
2018-01-01 17:00:00.097000-05:00,2675.00,9,24075.00
2018-01-01 17:00:00.097000-05:00,2675.00,1,2675.00
2018-01-01 17:00:00.097000-05:00,2675.25,15,40128.75
2018-01-01 17:00:00.097000-05:00,2675.50,10,26755.00


In [8]:
st.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30468167 entries, 2018-01-01 22:00:00.087000-05:00 to 2018-06-22 19:59:59.802000-04:00
Data columns (total 3 columns):
last      float32
volume    int16
dv        float32
dtypes: float32(2), int16(1)
memory usage: 523.0 MB
